In [1]:
import numpy as np
import pandas as pd
import os, sys
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')


%cd C:\emobility

C:\emobility


## Car assignment

In [47]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')


emission_factor = pd.read_csv(f'./dbs/emission/EmissionFactor.csv', delimiter=";")
PHEV = emission_factor[(emission_factor['carType']== 'Car_PHEV_bensin') | (emission_factor['carType']== 'Car_PHEV_diesel')].mean(axis=0)
emission_factor = emission_factor.append( PHEV, ignore_index=True ).drop([4, 5], axis=0)
emission_factor.iloc[5,0] = 'Car_PHEV'
emission_factor.replace(['Car_gasoline','Car_diesel','Car_ethanol','Car_gas','Car_el ','Car_PHEV'],[0,1,4,5,2,3], inplace=True)




car_distribution = pd.read_csv(f'./dbs/emission/CarDistribution.csv', delimiter=";")
car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']] = car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']].div(car_distribution['Total'], axis=0)

car_distribution = car_distribution.drop(['Total'], axis=1)



In [3]:
def carType_assignment(row, car_distribution):
    if row['deso'][:2] == '14':
        prob = car_distribution[['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']][car_distribution.Kod==int(row['deso'][:4])].values.tolist()
        pred = np.random.multinomial(1, prob[0], 1)
        carType = pred[0].tolist().index(1)
    else:
        # for those not in VG, use VG general distribution
        prob = [0.579957, 0.354470, 0.003693, 0.006844, 0.043670, 0.011365]
        pred = np.random.multinomial(1, prob, 1)
        carType = pred[0].tolist().index(1)

    return carType




In [5]:
#df_act = pd.merge(df_act, df_act_car_segments[["person", "carType"]], left_on= "person", right_on="person", how="left")

In [4]:
df_act_car_segments = df_act.drop_duplicates(subset=['person'])

df_act_car_segments['carType'] = 100

df_act_car_segments['carType'] = df_act_car_segments.apply( lambda row: carType_assignment(row, car_distribution), axis=1)

In [6]:
print('car distribution by fuel type at the aggregated level')
df_act_car_segments.carType.value_counts(normalize=True)

0    0.581296
1    0.353562
4    0.043068
5    0.011484
3    0.006796
2    0.003794
Name: carType, dtype: float64

## Emission calculation
### the road network classification by the population density in DeSO where the road link passes

In [96]:
import geopandas as gpd

#A DeSO in category A is rural areas. B is mostly located in a suburban areas. C is mostly located in the municipality's central location.
# read the road network and DeSO zones

road_network1 = gpd.read_file(f'./dbs/emission/road/5_road_network_with_slope.shp')

road_network = road_network[['link_id','geometry']]
road_network = road_network.reset_index()

DeSO = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Fordon_region.shp')
DeSO = DeSO[['Deso','geometry']]
DeSO['deso_category'] = DeSO['Deso'].str[4]


DeSO = DeSO.to_crs(road_network.crs)

In [97]:
from spatial_operations import spatial_join


if __name__ == '__main__':
    road_network = spatial_join(road_network, DeSO)

# the links crossing multiple DeSO are duplicated in the spatial join process.
# the link id is no unique, and it crosses multiple DeSO zones when it be combined

In [105]:

# road_network_zonecross= road_network[road_network.duplicated(subset=['index'], keep=False)]
#
#
# road_network_zonecross_mix = road_network_zonecross.groupby(['index']).agg({'deso_category': ' '.join}).reset_index()

In [122]:
# road_network_zonecross_mix.deso_category.value_counts()


In [106]:

# road_network_zonecross_mix = road_network_zonecross_mix[(road_network_zonecross_mix.deso_category=="C A")|(road_network_zonecross_mix.deso_category=="A C")]
#
#
#
# road_network = road_network.drop_duplicates(subset=['index'])
#
# road_network['deso_category'][road_network['index'].isin(list(road_network_zonecross_mix['index']))]='M'


In [115]:
road_network_Noduplicate = road_network.copy()

road_network_Noduplicate['deso_category'] = road_network_Noduplicate['deso_category'].fillna('M')


In [116]:
road_network_Noduplicate = road_network_Noduplicate.groupby(['link_id']).agg({'deso_category': ' '.join}).reset_index()

In [121]:
#road_network_Noduplicate.deso_category.value_counts()


In [118]:
def deso_categories(row):
    if len(row['deso_category'])==1:
        deso_category = row['deso_category']
    elif 'M' in row['deso_category']:
        deso_category = 'M'
    else:
        deso_category = "".join(set(row['deso_category']))
        if (len(deso_category)>1) & ("C" in deso_category) :
            deso_category = 'M'
        elif (len(deso_category)>1) :
            deso_category = 'B'
        #elif: (len(deso_category)>1) & ("A" in deso_category)  :
            #deso_category = 'B'

    return deso_category

In [104]:
# def deso_categories(row):
#     if len(row['deso_category'])==1:
#         deso_category = row['deso_category']
#     elif 'M' in row['deso_category']:
#         deso_category = 'M'
#     else:
#         deso_category = "".join(set(row['deso_category']))
#         if (len(deso_category)>1) & ("C" in deso_category) :
#             deso_category = 'M'
#         elif len(deso_category)>1  :
#             deso_category = 'B'
#
#     return deso_category


In [119]:
road_network_Noduplicate['deso_category'] = road_network_Noduplicate.apply( lambda row: deso_categories(row), axis=1)

In [120]:
#road_network_Noduplicate= pd.merge(road_network_Noduplicate, road_network[["link_id", "Deso"]], left_on= "link_id", right_on="link_id", how="left")
road_network_Noduplicate['deso_category'].value_counts(normalize=True)

C    0.455306
A    0.325817
B    0.200384
M    0.018493
Name: deso_category, dtype: float64

In [134]:
#road_network_Noduplicate = road_network_Noduplicate.groupby(['link_id']).agg({'deso_category': ' '.join}).reset_index()
# road_network_Deso = road_network.copy()
# road_network_Deso['Deso'] = road_network_Deso['Deso'].fillna('M')
# road_network_Deso = road_network_Deso.groupby(['link_id']).agg({'Deso': ' '.join}).reset_index()

In [137]:
road_network_Deso = road_network.copy()
road_network_Deso['Deso'] = road_network_Deso['Deso'].fillna('M')
road_network_Deso = road_network_Deso.groupby('link_id')['Deso'].apply(list).reset_index()


In [140]:
#len(road_network_Deso['Deso'].iloc[43])

3

### car movement trajectories on the road network

In [15]:
df_act_car_tra = pd.read_csv(f'./dbs/emission/paraset0.csv', delimiter=",")


In [16]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [17]:
df_act_car_tra = pd.merge(df_act_car_tra[['person', 'seq', 'time', 'purpose', 'type', 'link', 'distance_driven']], road_network_Noduplicate[['link_id', 'deso_category']], left_on= "link", right_on="link_id", how="left")

In [ ]:
df_act_car_tra = pd.merge(df_act_car_tra, road_network_Deso, left_on= "link", right_on="link_id", how="left")

In [18]:
#aaa = df_act_car_tra.head(10000)


In [19]:
#df_act_car_tra[df_act_car_tra.person==1971397]


,person,seq,time,purpose,type,link,distance_driven,link_id,deso_category
6955294,1971397,0,37122.0,NaN,vehicle enters traffic,1342685760006f,0.000000,1342685760006f,C
6955295,1971397,1,37123.0,NaN,left link,1342685760006f,0.000000,1342685760006f,C
6955296,1971397,2,37126.0,NaN,left link,7935409950004f,0.020595,7935409950004f,C
6955297,1971397,3,37134.0,NaN,left link,1053434250005f,0.094950,1053434250005f,C
6955298,1971397,4,37139.0,NaN,left link,1342686220020f,0.113064,1342686220020f,C
...,...,...,...,...,...,...,...,...,...
6956516,1971397,1222,71969.0,NaN,left link,1342686220010f,470.036766,1342686220010f,C
6956517,1971397,1223,71976.0,NaN,left link,5473078170005f,470.104721,5473078170005f,C
6956518,1971397,1224,71979.0,NaN,left link,7935409940002f,470.120087,7935409940002f,C
6956519,1971397,1225,72007.0,NaN,left link,1342685760006r,470.326020,1342685760006r,C


In [20]:
df_act_car_tra['distance_driven_shift'] = df_act_car_tra['distance_driven'].shift(-1)
df_act_car_tra['person_shift'] = df_act_car_tra['person'].shift(-1).fillna(0).astype('int')

df_act_car_tra['link_distance']=0

In [21]:
df_act_car_tra['link_distance'] = df_act_car_tra.apply( lambda row: row['distance_driven_shift']-row['distance_driven'] if row['person'] == row['person_shift'] else 0, axis=1)

In [22]:
# According to link distance (I have checked in Arcmap), shift rows.
df_act_car_tra['link_distance'] = df_act_car_tra['link_distance'].shift(1).fillna(0)

### activity purpose deduction

In [23]:
df_OTP = pd.read_csv(f'./dbs/agents/od_pairs_vg.csv')

df_OTP_last_activity = df_OTP.drop_duplicates(subset=['person'], keep='last')


In [24]:
#df_OTP[df_OTP.person==10002928]


,person,act_id,act_purpose,lat_o,lng_o,lat_d,lng_d,origin,destination,Age,Gender,cycling_speed,part
809881,10002928,0,work,57.757010,12.053026,57.728697,11.996491,1480,1480,51,Male,19.655208,29
809882,10002928,1,other,57.728697,11.996491,57.719952,11.998955,1480,1480,51,Male,17.658943,29
809883,10002928,2,work,57.719952,11.998955,57.728697,11.996491,1480,1480,51,Male,19.655208,29
809884,10002928,3,home,57.728697,11.996491,57.757010,12.053026,1480,1480,51,Male,17.658943,29


In [25]:
df_act_last_activity = df_act_car_tra[df_act_car_tra.type == 'vehicle leaves traffic']

df_act_last_activity = df_act_last_activity.drop_duplicates(subset=['person','type'], keep='last')

In [26]:
df_act_last_activity = pd.merge(df_act_last_activity[['person','seq']], df_OTP_last_activity[["person", "act_purpose"]], left_on= "person", right_on="person", how="left")


In [27]:
df_act_car_tra = pd.merge(df_act_car_tra, df_act_last_activity, on= ["person",'seq'], how="left")

In [28]:
df_act_car_tra['purpose'] = np.where(pd.isna(df_act_car_tra.purpose), df_act_car_tra['act_purpose'], df_act_car_tra['purpose'])

In [29]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [30]:
df_act_car_tra['purpose'].bfill(axis ='rows', inplace=True)


In [31]:
df_act_car_tra.drop(['person_shift', 'distance_driven_shift','act_purpose', 'link_id'], inplace=True, axis=1)

here

In [61]:
#bbb = pd.merge(df_OTP, aaa, left_on= ["person",'act_id'], right_on=["person",'act_id'], how="left")

In [74]:
#df_act[df_act.person==2208394]

,person,act_id,deso,POINT_X,POINT_Y,act_purpose,mode,dep_time,trav_time,trav_time_min,speed,distance,act_start,act_time,act_end,score
312714,2208394,0,1481C1110,324980.6320,6.388701e+06,home,NaN,0.000000,00:00:00,0.000000,NaN,0.000000,0,653.316667,10.888611,45.413392
312715,2208394,1,1481C1140,321921.7193,6.391209e+06,other,pt,10.888611,00:17:20,17.333333,17.798488,5.141786,671,44.216667,11.914444,45.413392
312716,2208394,2,1481C1110,324980.6320,6.388701e+06,home,pt,11.914444,00:17:23,17.383333,17.747294,5.141786,732,94.266667,13.775278,45.413392
312717,2208394,3,1481C1070,324452.0205,6.385596e+06,other,car,13.775278,00:12:52,12.866667,31.272415,6.706196,839,52.433333,14.863611,45.413392
312718,2208394,4,1481C1300,321317.6230,6.394844e+06,other,car,14.863611,01:06:54,66.900000,11.449286,12.765954,959,11.466667,16.169722,45.413392
312719,2208394,5,1481C1110,324980.6320,6.388701e+06,home,car,16.169722,00:43:51,43.850000,12.706528,9.286354,1014,425.950000,23.999722,45.413392


In [86]:
#df_act_car_tra[df_act_car_tra.person==2208394]

here

### assign activity IDs

In [76]:
act_id = df_act_car_tra[df_act_car_tra.type == 'vehicle leaves traffic']
act_id['act_id'] = act_id.groupby('person').cumcount()

df_act_car_tra = pd.merge(df_act_car_tra, aaa[['act_id']], left_index=True, right_index=True, how="left")



IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [77]:
df_act_car_tra['act_id'].bfill(axis ='rows', inplace=True)
df_act_car_tra['act_id'] = df_act_car_tra['act_id'].astype('int')


In [79]:
df_act_car_tra_bycategories = df_act_car_tra.groupby(['person','act_id','deso_category'])['link_distance'].sum().reset_index()


In [80]:
### add car with the fuel type to the trajectories

df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, df_act_car_segments[["person", "carType"]], left_on= "person", right_on="person", how="left")

In [81]:
df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, emission_factor, left_on= "carType", right_on="carType", how="left")

### calculate the link emissions

In [53]:
def link_emission(row):
    if row['deso_category']=='C':
        link_emission = row['link_distance']*row['TTW_city']
    elif row['deso_category']=='M':
        link_emission = row['link_distance']*row['TTW_mix']
    elif (row['deso_category']=='A')|(row['deso_category']=='B'):
        link_emission = row['link_distance']*row['TTW_rural']
    return link_emission

In [82]:
df_act_car_tra_bycategories['link_emission'] = df_act_car_tra_bycategories.apply( lambda row: link_emission(row), axis=1)

In [83]:
df_act_car_tra_bycategories['link_emission'].sum()

2498299.6964397253

In [84]:
df_act_car_byactivities = df_act_car_tra_bycategories.groupby(['person','act_id'])['link_emission'].sum().reset_index()


2498299.6964397235